In [32]:
import os, random, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report, precision_recall_curve
from joblib import dump
from xgboost import XGBClassifier
import warnings
import joblib
warnings.filterwarnings('ignore')

# ---- CONFIGURATION ----
CONFIG = {
    'test_size': 0.20,
    'val_size': 0.20,
    'random_state': 42,
    'early_stopping_rounds': 100,
    'n_estimators': 2000,
    'learning_rate': 0.03,
    'max_depth': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'threshold_precision': 99,  # Number of thresholds to test
    'min_samples_leaf': 3,      # Prevent overfitting
    'reg_alpha': 0.1,           # L1 regularization
    'reg_lambda': 1.0           # L2 regularization
}

def validate_data(df):
    """Validate input data structure and quality"""
    assert 'df' in globals() or df is not None, "DataFrame not found"
    assert 'result' in df.columns, "Target column 'result' not found"

    # Check for data quality issues
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"Dataset shape: {df.shape}")
    print(f"Numeric features: {len(numeric_cols)-1}")  # -1 for target
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Class distribution: {df['result'].value_counts().to_dict()}")

    return df

def setup_reproducibility(seed=42):
    """Set all random seeds for reproducibility"""
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

def prepare_features_target(df):
    """Prepare features and target with enhanced preprocessing"""
    y = df['result'].astype(int)

    # Select numeric features and handle potential issues
    X = df.drop(columns=['result']).select_dtypes(include=[np.number]).copy()

    # Handle infinite values
    X = X.replace([np.inf, -np.inf], np.nan)

    # Fill missing values with median (more robust than mean)
    X = X.fillna(X.median())

    # Remove constant features (no predictive power)
    constant_features = X.columns[X.nunique() <= 1]
    if len(constant_features) > 0:
        print(f"Removing {len(constant_features)} constant features")
        X = X.drop(columns=constant_features)

    print(f"Final feature count: {X.shape[1]}")
    return X, y

def create_splits(X, y, config):
    """Create train/validation/test splits"""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=config['test_size'],
        stratify=y, random_state=config['random_state']
    )

    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=config['val_size'],
        stratify=y_train, random_state=config['random_state']
    )

    return X_tr, X_val, X_test, y_tr, y_val, y_test

def calculate_class_weights(y_train):
    """Calculate class weights for imbalanced data"""
    pos = (y_train == 1).sum()
    neg = (y_train == 0).sum()
    assert pos > 0 and neg > 0, "Both classes must be present"

    ratio = neg / pos
    print(f"Class imbalance ratio (neg/pos): {ratio:.2f}")
    return ratio

def train_model(X_tr, y_tr, X_val, y_val, scale_pos_weight, config):
    """Train XGBoost model with optimized parameters"""
    # Base model without early stopping parameters
    model = XGBClassifier(
        random_state=config['random_state'],
        n_jobs=-1,
        tree_method="hist",
        scale_pos_weight=scale_pos_weight,
        eval_metric="logloss",
        n_estimators=config['n_estimators'],
        learning_rate=config['learning_rate'],
        max_depth=config['max_depth'],
        subsample=config['subsample'],
        colsample_bytree=config['colsample_bytree'],
        min_child_weight=config['min_samples_leaf'],
        reg_alpha=config['reg_alpha'],
        reg_lambda=config['reg_lambda']
    )

    print("Training model with early stopping...")

    # Try different early stopping approaches based on XGBoost version
    fit_params = {
        'X': X_tr,
        'y': y_tr,
        'eval_set': [(X_val, y_val)],
        'verbose': False
    }

    try:
        # Method 1: Try new callback syntax (XGBoost >= 1.6.0)
        from xgboost.callback import EarlyStopping
        fit_params['callbacks'] = [EarlyStopping(
            rounds=config['early_stopping_rounds'],
            save_best=True,
            maximize=False
        )]
        model.fit(**fit_params)
        print("Used new callback syntax")

    except (TypeError, ImportError, AttributeError):
        try:
            # Method 2: Try legacy parameter in fit() (XGBoost 1.4-1.5)
            fit_params.pop('callbacks', None)  # Remove callbacks if present
            fit_params['early_stopping_rounds'] = config['early_stopping_rounds']
            model.fit(**fit_params)
            print("Used legacy fit parameter")

        except TypeError:
            # Method 3: Very old XGBoost - train without early stopping
            print("Early stopping not supported, training with full n_estimators...")
            fit_params = {'X': X_tr, 'y': y_tr}
            model.fit(**fit_params)

    # Get best iteration (handle different attribute names)
    best_iter = getattr(model, 'best_iteration',
                       getattr(model, 'best_ntree_limit',
                              getattr(model, 'n_estimators', 'N/A')))
    print(f"Training completed. Best iteration: {best_iter}")
    return model

def find_optimal_threshold(y_true, y_proba, config, metric='f1'):
    """Find optimal threshold using specified metric"""
    thresholds = np.linspace(0.01, 0.99, config['threshold_precision'])

    if metric == 'f1':
        scores = [f1_score(y_true, (y_proba >= t).astype(int)) for t in thresholds]
    elif metric == 'precision_recall':
        # Use precision-recall curve for better threshold selection
        precision, recall, pr_thresholds = precision_recall_curve(y_true, y_proba)
        # Find threshold that maximizes F1 score
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_idx = np.argmax(f1_scores)
        return float(pr_thresholds[best_idx] if best_idx < len(pr_thresholds) else 0.5)

    best_threshold = float(thresholds[np.argmax(scores)])
    best_score = max(scores)

    print(f"Best {metric} score: {best_score:.4f} at threshold: {best_threshold:.3f}")
    return best_threshold

def evaluate_model(model, X_val, y_val, X_test, y_test, threshold, config):
    """Comprehensive model evaluation"""
    # Validation metrics
    y_val_proba = model.predict_proba(X_val)[:, 1]
    val_ap = average_precision_score(y_val, y_val_proba)
    val_roc = roc_auc_score(y_val, y_val_proba)

    # Test metrics
    y_test_proba = model.predict_proba(X_test)[:, 1]
    test_ap = average_precision_score(y_test, y_test_proba)
    test_roc = roc_auc_score(y_test, y_test_proba)
    y_test_pred = (y_test_proba >= threshold).astype(int)

    print(f"Validation  AP: {val_ap:.4f} | ROC: {val_roc:.4f}")
    print(f"Test        AP: {test_ap:.4f} | ROC: {test_roc:.4f} | Threshold: {threshold:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_test_pred, digits=4))

    return y_test_proba, {
        'val_ap': val_ap, 'val_roc': val_roc,
        'test_ap': test_ap, 'test_roc': test_roc,
        'threshold': threshold
    }

def create_feature_importance(model, feature_names, output_dir="outputs"):
    """Create comprehensive feature importance analysis"""
    os.makedirs(output_dir, exist_ok=True)

    booster = model.get_booster()

    # Multiple importance types for better insight
    importance_types = ['gain', 'weight', 'cover']
    all_importance = {}

    for imp_type in importance_types:
        imp_dict = booster.get_score(importance_type=imp_type)
        feat_map = {f"f{i}": col for i, col in enumerate(feature_names)}
        all_importance[imp_type] = {feat_map.get(k, k): v for k, v in imp_dict.items()}

    # Create comprehensive importance DataFrame
    importance_data = []
    for feature in feature_names:
        row = {'feature': feature}
        for imp_type in importance_types:
            row[f'importance_{imp_type}'] = all_importance[imp_type].get(feature, 0)
        importance_data.append(row)

    fi_df = pd.DataFrame(importance_data).sort_values('importance_gain', ascending=False)

    # Add normalized importance (0-100 scale)
    for imp_type in importance_types:
        col = f'importance_{imp_type}'
        fi_df[f'{col}_normalized'] = (fi_df[col] / fi_df[col].max() * 100).round(2)

    fi_df.to_csv(f"{output_dir}/feature_importance_detailed.csv", index=False)

    # Top 20 for Power BI (easier visualization)
    fi_top20 = fi_df.head(20)[['feature', 'importance_gain_normalized']].copy()
    fi_top20.to_csv(f"{output_dir}/feature_importance_top20.csv", index=False)

    print(f"Feature importance saved to {output_dir}/")
    return fi_df

def create_powerbi_outputs(model, X_test, y_test, y_test_proba, threshold, metrics, output_dir="outputs"):
    """Create optimized outputs for Power BI visualization"""
    os.makedirs(output_dir, exist_ok=True)

    # 1. Enhanced predictions with more granular risk categories
    risk_bins = [0, threshold*0.4, threshold*0.7, threshold, threshold*1.5, 1.01]
    risk_labels = ["Very Low", "Low", "Medium", "High", "Critical"]
    risk_category = pd.cut(y_test_proba, bins=risk_bins, labels=risk_labels, include_lowest=True)

    # Create comprehensive predictions dataset
    predictions_df = pd.DataFrame({
        'row_index': X_test.index,
        'actual_result': y_test.values,
        'probability_default': np.round(y_test_proba, 4),
        'predicted_result': (y_test_proba >= threshold).astype(int),
        'risk_category': risk_category,
        'risk_score': np.round(y_test_proba * 100, 1),  # 0-100 scale
        'confidence_level': pd.cut(y_test_proba,
                                 bins=[0, 0.3, 0.7, 1.01],
                                 labels=["Low", "Medium", "High"])
    })

    # Add decile analysis for better segmentation
    predictions_df['risk_decile'] = pd.qcut(y_test_proba, 10, labels=range(1, 11))

    predictions_df.to_csv(f"{output_dir}/predictions_enhanced.csv", index=False)

    # 2. Model performance summary for dashboard KPIs
    performance_summary = pd.DataFrame([{
        'metric': 'Precision-Recall AUC',
        'validation_score': round(metrics['val_ap'], 4),
        'test_score': round(metrics['test_ap'], 4),
        'description': 'Area under precision-recall curve (primary metric)'
    }, {
        'metric': 'ROC AUC',
        'validation_score': round(metrics['val_roc'], 4),
        'test_score': round(metrics['test_roc'], 4),
        'description': 'Area under ROC curve'
    }, {
        'metric': 'Optimal Threshold',
        'validation_score': round(threshold, 3),
        'test_score': round(threshold, 3),
        'description': 'Threshold maximizing F1 score'
    }])

    performance_summary.to_csv(f"{output_dir}/model_performance_summary.csv", index=False)

    # 3. Risk distribution analysis
    risk_analysis = predictions_df.groupby('risk_category').agg({
        'probability_default': ['count', 'mean', 'std'],
        'actual_result': ['sum', 'mean']
    }).round(4)

    risk_analysis.columns = ['_'.join(col).strip() for col in risk_analysis.columns]
    risk_analysis = risk_analysis.reset_index()
    risk_analysis['default_rate_actual'] = (risk_analysis['actual_result_sum'] /
                                          risk_analysis['probability_default_count'] * 100).round(2)

    risk_analysis.to_csv(f"{output_dir}/risk_category_analysis.csv", index=False)

    # 4. Decile analysis for model calibration
    decile_analysis = predictions_df.groupby('risk_decile').agg({
        'probability_default': ['count', 'min', 'max', 'mean'],
        'actual_result': ['sum', 'mean']
    }).round(4)

    decile_analysis.columns = ['_'.join(col).strip() for col in decile_analysis.columns]
    decile_analysis = decile_analysis.reset_index()
    decile_analysis['default_rate_actual'] = (decile_analysis['actual_result_sum'] /
                                            decile_analysis['probability_default_count'] * 100).round(2)
    decile_analysis['default_rate_predicted'] = (decile_analysis['probability_default_mean'] * 100).round(2)

    decile_analysis.to_csv(f"{output_dir}/decile_analysis.csv", index=False)

    print(f"\nPower BI outputs created in '{output_dir}/' directory:")
    print("- predictions_enhanced.csv (main predictions with risk categories)")
    print("- model_performance_summary.csv (KPI metrics)")
    print("- risk_category_analysis.csv (risk distribution)")
    print("- decile_analysis.csv (model calibration)")

    return predictions_df

def save_model_artifacts(model, threshold, feature_names, scale_pos_weight, metrics, output_dir="outputs"):
    """Save model and metadata for production deployment"""
    os.makedirs(output_dir, exist_ok=True)

    # Enhanced model bundle
    model_bundle = {
        'model': model,
        'threshold': threshold,
        'features': list(feature_names),
        'scale_pos_weight': scale_pos_weight,
        'performance_metrics': metrics,
        'model_config': CONFIG,
        'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'feature_count': len(feature_names)
    }

    dump(model_bundle, f"{output_dir}/bankruptcy_model_complete.joblib")

    # Create model metadata for documentation
    metadata_df = pd.DataFrame([{
        'parameter': k,
        'value': str(v)
    } for k, v in CONFIG.items()])

    metadata_df.to_csv(f"{output_dir}/model_metadata.csv", index=False)
    print(f"Model artifacts saved to {output_dir}/")

# ---- MAIN EXECUTION ----
def main():
    # Data validation
    df = joblib.load('/content/drive/MyDrive/Bankruptcy Prediction Data/Prepared Dataset/Dataset.pkl')

    df = validate_data(df)  # Assumes df exists in global scope

    # Setup reproducibility
    setup_reproducibility(CONFIG['random_state'])

    # Prepare data
    X, y = prepare_features_target(df)

    # Create splits
    X_tr, X_val, X_test, y_tr, y_val, y_test = create_splits(X, y, CONFIG)

    # Calculate class weights
    scale_pos_weight = calculate_class_weights(y_tr)

    # Train model
    model = train_model(X_tr, y_tr, X_val, y_val, scale_pos_weight, CONFIG)

    # Find optimal threshold (using precision-recall curve for better results)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    threshold = find_optimal_threshold(y_val, y_val_proba, CONFIG, metric='precision_recall')

    # Evaluate model
    y_test_proba, metrics = evaluate_model(model, X_val, y_val, X_test, y_test, threshold, CONFIG)

    # Create feature importance analysis
    fi_df = create_feature_importance(model, X.columns)

    # Create Power BI optimized outputs
    predictions_df = create_powerbi_outputs(model, X_test, y_test, y_test_proba, threshold, metrics)

    # Save complete model artifacts
    save_model_artifacts(model, threshold, X.columns, scale_pos_weight, metrics)

    return model, threshold, predictions_df, fi_df, metrics

# ---- EXECUTION ----
if __name__ == "__main__" or 'df' in globals():
    try:
        model, threshold, predictions_df, fi_df, metrics = main()
        print("\n✅ Pipeline completed successfully!")
        print(f"📊 Check the 'outputs/' folder for Power BI ready files")

    except Exception as e:
        print(f"❌ Pipeline failed: {str(e)}")
        raise

Dataset shape: (255919, 61)
Numeric features: 60
Missing values: 0
Class distribution: {0: 227559, 1: 28360}
Final feature count: 60
Class imbalance ratio (neg/pos): 8.02
Training model with early stopping...
Early stopping not supported, training with full n_estimators...
Training completed. Best iteration: 2000
Validation  AP: 0.8597 | ROC: 0.9721
Test        AP: 0.8667 | ROC: 0.9739 | Threshold: 0.799

Classification Report:
              precision    recall  f1-score   support

           0     0.9731    0.9737    0.9734     45512
           1     0.7877    0.7844    0.7860      5672

    accuracy                         0.9527     51184
   macro avg     0.8804    0.8790    0.8797     51184
weighted avg     0.9526    0.9527    0.9526     51184

Feature importance saved to outputs/
❌ Pipeline failed: bins must increase monotonically.


ValueError: bins must increase monotonically.

In [33]:
import os, random, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, classification_report, precision_recall_curve
from joblib import dump
from xgboost import XGBClassifier
import warnings
import joblib
warnings.filterwarnings('ignore')

# ---- CONFIGURATION ----
CONFIG = {
    'test_size': 0.20,
    'val_size': 0.20,
    'random_state': 42,
    'early_stopping_rounds': 100,
    'n_estimators': 2000,
    'learning_rate': 0.03,
    'max_depth': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'threshold_precision': 99,  # Number of thresholds to test
    'min_samples_leaf': 3,      # Prevent overfitting
    'reg_alpha': 0.1,           # L1 regularization
    'reg_lambda': 1.0           # L2 regularization
}

def validate_data(df):
    """Validate input data structure and quality"""
    assert 'df' in globals() or df is not None, "DataFrame not found"
    assert 'result' in df.columns, "Target column 'result' not found"

    # Check for data quality issues
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"Dataset shape: {df.shape}")
    print(f"Numeric features: {len(numeric_cols)-1}")  # -1 for target
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Class distribution: {df['result'].value_counts().to_dict()}")

    return df

def setup_reproducibility(seed=42):
    """Set all random seeds for reproducibility"""
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

def prepare_features_target(df):
    """Prepare features and target with enhanced preprocessing"""
    y = df['result'].astype(int)

    # Select numeric features and handle potential issues
    X = df.drop(columns=['result']).select_dtypes(include=[np.number]).copy()

    # Handle infinite values
    X = X.replace([np.inf, -np.inf], np.nan)

    # Fill missing values with median (more robust than mean)
    X = X.fillna(X.median())

    # Remove constant features (no predictive power)
    constant_features = X.columns[X.nunique() <= 1]
    if len(constant_features) > 0:
        print(f"Removing {len(constant_features)} constant features")
        X = X.drop(columns=constant_features)

    print(f"Final feature count: {X.shape[1]}")
    return X, y

def create_splits(X, y, config):
    """Create train/validation/test splits"""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=config['test_size'],
        stratify=y, random_state=config['random_state']
    )

    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=config['val_size'],
        stratify=y_train, random_state=config['random_state']
    )

    return X_tr, X_val, X_test, y_tr, y_val, y_test

def calculate_class_weights(y_train):
    """Calculate class weights for imbalanced data"""
    pos = (y_train == 1).sum()
    neg = (y_train == 0).sum()
    assert pos > 0 and neg > 0, "Both classes must be present"

    ratio = neg / pos
    print(f"Class imbalance ratio (neg/pos): {ratio:.2f}")
    return ratio

def train_model(X_tr, y_tr, X_val, y_val, scale_pos_weight, config):
    """Train XGBoost model with optimized parameters"""
    # Base model without early stopping parameters
    model = XGBClassifier(
        random_state=config['random_state'],
        n_jobs=-1,
        tree_method="hist",
        scale_pos_weight=scale_pos_weight,
        eval_metric="logloss",
        n_estimators=config['n_estimators'],
        learning_rate=config['learning_rate'],
        max_depth=config['max_depth'],
        subsample=config['subsample'],
        colsample_bytree=config['colsample_bytree'],
        min_child_weight=config['min_samples_leaf'],
        reg_alpha=config['reg_alpha'],
        reg_lambda=config['reg_lambda']
    )

    print("Training model with early stopping...")

    # Try different early stopping approaches based on XGBoost version
    fit_params = {
        'X': X_tr,
        'y': y_tr,
        'eval_set': [(X_val, y_val)],
        'verbose': False
    }

    try:
        # Method 1: Try new callback syntax (XGBoost >= 1.6.0)
        from xgboost.callback import EarlyStopping
        fit_params['callbacks'] = [EarlyStopping(
            rounds=config['early_stopping_rounds'],
            save_best=True,
            maximize=False
        )]
        model.fit(**fit_params)
        print("Used new callback syntax")

    except (TypeError, ImportError, AttributeError):
        try:
            # Method 2: Try legacy parameter in fit() (XGBoost 1.4-1.5)
            fit_params.pop('callbacks', None)  # Remove callbacks if present
            fit_params['early_stopping_rounds'] = config['early_stopping_rounds']
            model.fit(**fit_params)
            print("Used legacy fit parameter")

        except TypeError:
            # Method 3: Very old XGBoost - train without early stopping
            print("Early stopping not supported, training with full n_estimators...")
            fit_params = {'X': X_tr, 'y': y_tr}
            model.fit(**fit_params)

    # Get best iteration (handle different attribute names)
    best_iter = getattr(model, 'best_iteration',
                       getattr(model, 'best_ntree_limit',
                              getattr(model, 'n_estimators', 'N/A')))
    print(f"Training completed. Best iteration: {best_iter}")
    return model

def find_optimal_threshold(y_true, y_proba, config, metric='f1'):
    """Find optimal threshold using specified metric"""
    thresholds = np.linspace(0.01, 0.99, config['threshold_precision'])

    if metric == 'f1':
        scores = [f1_score(y_true, (y_proba >= t).astype(int)) for t in thresholds]
    elif metric == 'precision_recall':
        # Use precision-recall curve for better threshold selection
        precision, recall, pr_thresholds = precision_recall_curve(y_true, y_proba)
        # Find threshold that maximizes F1 score
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_idx = np.argmax(f1_scores)
        return float(pr_thresholds[best_idx] if best_idx < len(pr_thresholds) else 0.5)

    best_threshold = float(thresholds[np.argmax(scores)])
    best_score = max(scores)

    print(f"Best {metric} score: {best_score:.4f} at threshold: {best_threshold:.3f}")
    return best_threshold

def evaluate_model(model, X_val, y_val, X_test, y_test, threshold, config):
    """Comprehensive model evaluation"""
    # Validation metrics
    y_val_proba = model.predict_proba(X_val)[:, 1]
    val_ap = average_precision_score(y_val, y_val_proba)
    val_roc = roc_auc_score(y_val, y_val_proba)

    # Test metrics
    y_test_proba = model.predict_proba(X_test)[:, 1]
    test_ap = average_precision_score(y_test, y_test_proba)
    test_roc = roc_auc_score(y_test, y_test_proba)
    y_test_pred = (y_test_proba >= threshold).astype(int)

    print(f"Validation  AP: {val_ap:.4f} | ROC: {val_roc:.4f}")
    print(f"Test        AP: {test_ap:.4f} | ROC: {test_roc:.4f} | Threshold: {threshold:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_test_pred, digits=4))

    return y_test_proba, {
        'val_ap': val_ap, 'val_roc': val_roc,
        'test_ap': test_ap, 'test_roc': test_roc,
        'threshold': threshold
    }

def create_feature_importance(model, feature_names, output_dir="outputs"):
    """Create comprehensive feature importance analysis"""
    os.makedirs(output_dir, exist_ok=True)

    booster = model.get_booster()

    # Multiple importance types for better insight
    importance_types = ['gain', 'weight', 'cover']
    all_importance = {}

    for imp_type in importance_types:
        imp_dict = booster.get_score(importance_type=imp_type)
        feat_map = {f"f{i}": col for i, col in enumerate(feature_names)}
        all_importance[imp_type] = {feat_map.get(k, k): v for k, v in imp_dict.items()}

    # Create comprehensive importance DataFrame
    importance_data = []
    for feature in feature_names:
        row = {'feature': feature}
        for imp_type in importance_types:
            row[f'importance_{imp_type}'] = all_importance[imp_type].get(feature, 0)
        importance_data.append(row)

    fi_df = pd.DataFrame(importance_data).sort_values('importance_gain', ascending=False)

    # Add normalized importance (0-100 scale)
    for imp_type in importance_types:
        col = f'importance_{imp_type}'
        fi_df[f'{col}_normalized'] = (fi_df[col] / fi_df[col].max() * 100).round(2)

    fi_df.to_csv(f"{output_dir}/feature_importance_detailed.csv", index=False)

    # Top 20 for Power BI (easier visualization)
    fi_top20 = fi_df.head(20)[['feature', 'importance_gain_normalized']].copy()
    fi_top20.to_csv(f"{output_dir}/feature_importance_top20.csv", index=False)

    print(f"Feature importance saved to {output_dir}/")
    return fi_df

def create_powerbi_outputs(model, X_test, y_test, y_test_proba, threshold, metrics, output_dir="outputs"):
    """Create optimized outputs for Power BI visualization"""
    os.makedirs(output_dir, exist_ok=True)

    # 1. Enhanced predictions with more granular risk categories
    # Create bins ensuring they are monotonically increasing and within [0,1]
    bin1 = min(0.2, threshold * 0.4)
    bin2 = min(0.4, threshold * 0.7)
    bin3 = threshold
    bin4 = min(0.9, max(threshold * 1.2, threshold + 0.1))  # Ensure it's above threshold

    # Ensure monotonic increasing and valid bounds
    risk_bins = [0, bin1, bin2, bin3, bin4, 1.0]
    risk_bins = sorted(list(set(risk_bins)))  # Remove duplicates and sort

    # Adjust labels based on actual number of bins
    if len(risk_bins) == 6:
        risk_labels = ["Very Low", "Low", "Medium", "High", "Critical"]
    elif len(risk_bins) == 5:
        risk_labels = ["Low", "Medium", "High", "Critical"]
    elif len(risk_bins) == 4:
        risk_labels = ["Low", "Medium", "High"]
    else:
        risk_labels = ["Low", "High"]

    print(f"Risk bins: {[round(b, 3) for b in risk_bins]}")
    risk_category = pd.cut(y_test_proba, bins=risk_bins, labels=risk_labels[:len(risk_bins)-1], include_lowest=True)
    # Create comprehensive predictions dataset
    predictions_df = pd.DataFrame({
        'row_index': X_test.index,
        'actual_result': y_test.values,
        'probability_default': np.round(y_test_proba, 4),
        'predicted_result': (y_test_proba >= threshold).astype(int),
        'risk_category': risk_category,
        'risk_score': np.round(y_test_proba * 100, 1),  # 0-100 scale
        'confidence_level': pd.cut(y_test_proba,
                                 bins=[0, 0.3, 0.7, 1.0],
                                 labels=["Low", "Medium", "High"],
                                 include_lowest=True)
    })

    # Add decile analysis for better segmentation
    predictions_df['risk_decile'] = pd.qcut(y_test_proba, 10, labels=range(1, 11), duplicates='drop')

    predictions_df.to_csv(f"{output_dir}/predictions_enhanced.csv", index=False)

    # 2. Model performance summary for dashboard KPIs
    performance_summary = pd.DataFrame([{
        'metric': 'Precision-Recall AUC',
        'validation_score': round(metrics['val_ap'], 4),
        'test_score': round(metrics['test_ap'], 4),
        'description': 'Area under precision-recall curve (primary metric)'
    }, {
        'metric': 'ROC AUC',
        'validation_score': round(metrics['val_roc'], 4),
        'test_score': round(metrics['test_roc'], 4),
        'description': 'Area under ROC curve'
    }, {
        'metric': 'Optimal Threshold',
        'validation_score': round(threshold, 3),
        'test_score': round(threshold, 3),
        'description': 'Threshold maximizing F1 score'
    }])

    performance_summary.to_csv(f"{output_dir}/model_performance_summary.csv", index=False)

    # 3. Risk distribution analysis
    risk_analysis = predictions_df.groupby('risk_category').agg({
        'probability_default': ['count', 'mean', 'std'],
        'actual_result': ['sum', 'mean']
    }).round(4)

    risk_analysis.columns = ['_'.join(col).strip() for col in risk_analysis.columns]
    risk_analysis = risk_analysis.reset_index()
    risk_analysis['default_rate_actual'] = (risk_analysis['actual_result_sum'] /
                                          risk_analysis['probability_default_count'] * 100).round(2)

    risk_analysis.to_csv(f"{output_dir}/risk_category_analysis.csv", index=False)

    # 4. Decile analysis for model calibration
    decile_analysis = predictions_df.groupby('risk_decile').agg({
        'probability_default': ['count', 'min', 'max', 'mean'],
        'actual_result': ['sum', 'mean']
    }).round(4)

    decile_analysis.columns = ['_'.join(col).strip() for col in decile_analysis.columns]
    decile_analysis = decile_analysis.reset_index()
    decile_analysis['default_rate_actual'] = (decile_analysis['actual_result_sum'] /
                                            decile_analysis['probability_default_count'] * 100).round(2)
    decile_analysis['default_rate_predicted'] = (decile_analysis['probability_default_mean'] * 100).round(2)

    decile_analysis.to_csv(f"{output_dir}/decile_analysis.csv", index=False)

    print(f"\nPower BI outputs created in '{output_dir}/' directory:")
    print("- predictions_enhanced.csv (main predictions with risk categories)")
    print("- model_performance_summary.csv (KPI metrics)")
    print("- risk_category_analysis.csv (risk distribution)")
    print("- decile_analysis.csv (model calibration)")

    return predictions_df

def save_model_artifacts(model, threshold, feature_names, scale_pos_weight, metrics, output_dir="outputs"):
    """Save model and metadata for production deployment"""
    os.makedirs(output_dir, exist_ok=True)

    # Enhanced model bundle
    model_bundle = {
        'model': model,
        'threshold': threshold,
        'features': list(feature_names),
        'scale_pos_weight': scale_pos_weight,
        'performance_metrics': metrics,
        'model_config': CONFIG,
        'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'feature_count': len(feature_names)
    }

    dump(model_bundle, f"{output_dir}/bankruptcy_model_complete.joblib")

    # Create model metadata for documentation
    metadata_df = pd.DataFrame([{
        'parameter': k,
        'value': str(v)
    } for k, v in CONFIG.items()])

    metadata_df.to_csv(f"{output_dir}/model_metadata.csv", index=False)
    print(f"Model artifacts saved to {output_dir}/")

# ---- MAIN EXECUTION ----
def main():
    # Data validation
    df = joblib.load('/content/drive/MyDrive/Bankruptcy Prediction Data/Prepared Dataset/Dataset.pkl')

    df = validate_data(df)  # Assumes df exists in global scope

    # Setup reproducibility
    setup_reproducibility(CONFIG['random_state'])

    # Prepare data
    X, y = prepare_features_target(df)

    # Create splits
    X_tr, X_val, X_test, y_tr, y_val, y_test = create_splits(X, y, CONFIG)

    # Calculate class weights
    scale_pos_weight = calculate_class_weights(y_tr)

    # Train model
    model = train_model(X_tr, y_tr, X_val, y_val, scale_pos_weight, CONFIG)

    # Find optimal threshold (using precision-recall curve for better results)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    threshold = find_optimal_threshold(y_val, y_val_proba, CONFIG, metric='precision_recall')

    # Evaluate model
    y_test_proba, metrics = evaluate_model(model, X_val, y_val, X_test, y_test, threshold, CONFIG)

    # Create feature importance analysis
    fi_df = create_feature_importance(model, X.columns)

    # Create Power BI optimized outputs
    predictions_df = create_powerbi_outputs(model, X_test, y_test, y_test_proba, threshold, metrics)

    # Save complete model artifacts
    save_model_artifacts(model, threshold, X.columns, scale_pos_weight, metrics)

    return model, threshold, predictions_df, fi_df, metrics

# ---- EXECUTION ----
if __name__ == "__main__" or 'df' in globals():
    try:
        model, threshold, predictions_df, fi_df, metrics = main()
        print("\n✅ Pipeline completed successfully!")
        print(f"📊 Check the 'outputs/' folder for Power BI ready files")

    except Exception as e:
        print(f"❌ Pipeline failed: {str(e)}")
        raise

Dataset shape: (255919, 61)
Numeric features: 60
Missing values: 0
Class distribution: {0: 227559, 1: 28360}
Final feature count: 60
Class imbalance ratio (neg/pos): 8.02
Training model with early stopping...
Early stopping not supported, training with full n_estimators...
Training completed. Best iteration: 2000
Validation  AP: 0.8597 | ROC: 0.9721
Test        AP: 0.8667 | ROC: 0.9739 | Threshold: 0.799

Classification Report:
              precision    recall  f1-score   support

           0     0.9731    0.9737    0.9734     45512
           1     0.7877    0.7844    0.7860      5672

    accuracy                         0.9527     51184
   macro avg     0.8804    0.8790    0.8797     51184
weighted avg     0.9526    0.9527    0.9526     51184

Feature importance saved to outputs/
Risk bins: [0, 0.2, 0.4, 0.799, 0.9, 1.0]

Power BI outputs created in 'outputs/' directory:
- predictions_enhanced.csv (main predictions with risk categories)
- model_performance_summary.csv (KPI metrics